In [1]:
import numpy as np
from selenium import webdriver
from time import sleep
import pandas as pd
import threading
import os
from queue import Queue
import re

In [2]:
os.environ['PATH'] += r"/usr/local/bin/"

In [3]:
base_url = "https://batdongsan.vn/ban-nha/"

In [4]:
def openMultiBrowser(n):
    drivers = []
    for i in range(n):
        driver = webdriver.Firefox()
        drivers.append(driver)
    return drivers

def loadMultiPages(driver,n):
    driver.maximize_window()
    driver.get(f"{base_url}p{n}")
    sleep(5)
   
num_pages = [400,410,420,430,440]
def loadMultiBrowsers(drivers_rx): 
    for i,driver in enumerate(drivers_rx):
        t = threading.Thread(target=loadMultiPages,args=(driver,num_pages[i]))
        t.start()

In [5]:
def get_params(text):
    reg_template = {"Area":'Diện tích:\s*(\d+)',
                "Room":'Phòng ngủ:\s*(\d+)',
                "WC":'Phòng WC:\s*(\d+)',
                "Location":'Địa chỉ:\s*(.+)'}
    list_params = []
    for name in reg_template:
        x = re.search(r'{}'.format(reg_template[name]),text)
        if x:
            list_params.append(x.group(1))
        else:
            list_params.append(np.nan)
    return list_params


def get_data(driver):
    report = pd.DataFrame(columns=['Title', 'Price', 'Links',\
            'Diện tích (m2)', 'Số phòng ngủ','Số phòng WC', 'Địa chỉ'\
            ,'Description'])
    n_iter = 10
    for i in range(n_iter):
        driver.implicitly_wait(1)
        try:
            next = driver.find_element('css selector','.uk-pagination [rel="next"]').get_attribute('href')
            elements = driver.find_elements('css selector','.uk-width-medium-7-10 .name [href]')
            elements_price = driver.find_elements('css selector','.price')
        except:
            continue
            
        title = [element.text for element in elements]

        links = [element.get_attribute('href') for element in elements]

        price = [element.text for element in elements_price]
            
        df1 = pd.DataFrame(zip(title,price,links),columns=['Title','Price','Links'])
        
        data = []
        
        n = len(links)

        for j in range(n):
            try:
                driver.get(links[j])
                # sleep(5)
                params = driver.find_element('css selector',".param")
                content = driver.find_element('css selector',".content")
            except:
                df1.drop(j,axis=0,inplace=True)
                continue
            params_ = get_params(params.text)
            params_.append(content.text)
            data.append(params_)

        tmp = np.vstack([item for item in data])
        df2 = pd.DataFrame(tmp,columns=['Diện tích (m2)','Số phòng ngủ','Số phòng WC','Địa chỉ', 'Description'])
        df1 = pd.concat([df1,df2],axis=1)
        report = pd.concat([report,df1],axis=0)
        report.set_index(np.arange(len(report)),inplace=True)
        driver.get(next)
    return report


In [6]:
def runInParallel(func,drivers_rx):
    que = Queue()
    for driver in drivers_rx:
        print("------Running parallel------")
        t1 = threading.Thread(target= lambda q,arg1:q.put(func(arg1)),args=(que,driver))
        t1.start()
    ans = []
    for i in range(len(drivers_rx)):
        try:
            tmp = que.get()
            ans.append(tmp)
        except:
            continue
    print("------Done!------")
    return ans

In [7]:
driver_r1 = openMultiBrowser(5)
loadMultiBrowsers(driver_r1)
sleep(5)

In [8]:
list_report = runInParallel(get_data,driver_r1)

------Running parallel------
------Running parallel------
------Running parallel------
------Running parallel------
------Running parallel------
------Done!------


In [11]:
df = pd.concat([report for report in list_report],axis=0)
df

,Title,Price,Links,Diện tích (m2),Số phòng ngủ,Số phòng WC,Địa chỉ,Description
0,"Bán nhà phố Lĩnh Nam, 30m2 5 tầng mt 5.4m, nhỉ...",Giá bán\nDưới 500 triệu\nTừ 500 - 800 triệu\nT...,https://batdongsan.vn/ban-nha-pho-linh-nam-30m...,30,nan,nan,nan,"+ Ngõ 649 Lĩnh Nam to như phố, thông ra Nam Dư..."
1,"Bán nhà phố Đông Thiên, 31m2 3 ngủ, 2.95 tỷ có...",3.35 tỷ,https://batdongsan.vn/ban-nha-pho-dong-thien-3...,31,nan,nan,nan,+ Nhà mình bán ba gác đỗ cửa. Trước nhà thoáng...
2,"Bán nhà phố Lê Thanh Nghị, lô góc, dt 45m2 mt ...",2.95 tỷ,https://batdongsan.vn/ban-nha-pho-le-thanh-ngh...,45,nan,nan,nan,"+ Vị trí quá hiếm nhà bán, nằm trọn giữa quần ..."
3,"Bán nhà phố Nguyễn Khoái, 58m2 mt 4.5m, 10 ngủ...",4.95 tỷ,https://batdongsan.vn/ban-nha-pho-nguyen-khoai...,58,nan,nan,nan,"+ Nhà sau mặt phố Nguyễn Khoái 1 nhà, vài bước..."
4,"Tân Bình, hẻm xe tải thông, Lạc Long Quân, 64m...",6.4 tỷ,https://batdongsan.vn/tan-binh-hem-xe-tai-thon...,64,nan,nan,nan,"* Nhà 4 phòng ngủ, 4 toilet, phòng khách, bếp,..."
...,...,...,...,...,...,...,...,...
195,"6.x tỷ sở hữu ngay căn biệt thự siêu đẹp, đáng...",5900000 tỷ,https://batdongsan.vn/6x-ty-so-huu-ngay-can-bi...,135,nan,nan,"Hữu Nghị, Xã Phù Chẩn, Tp. Từ Sơn, Bắc Ninh, -...",????????căn biệt thự song lập 135 m2 tại KĐT V...
196,"Hoa Hậu Phân Lô, Quận Thanh Xuân, Gần phố, nhà...",6.9 tỷ,https://batdongsan.vn/hoa-hau-phan-lo-quan-tha...,64,4,3,Thanh Xuân,+ Chủ nhà người quen cần bán căn nhà Phường Ma...
197,HẺM XE HƠI 4M5 - TRUNG TÂM GÒ VẤP - 4X12M - CH...,7.15 tỷ,https://batdongsan.vn/hem-xe-hoi-4m5-trung-tam...,nan,2,2,nan,"Mô tả:\n- Diện tích vuông A4, xây hết đất\n- K..."
198,"Nhà bán, 1 tầng 1 trệt, 2PN, 2 WC, giá 4.75 tỷ",Thỏa thuận,https://batdongsan.vn/nha-ban-1-tang-1-tret-2p...,48,2,2,nan,"Mô tả:\n\n????Đường trước nhà: 2.5m trải nhựa,..."


In [14]:
df.to_csv('next_1000(2).csv',sep='\t')
